<a href="https://colab.research.google.com/github/vasudevgupta7/bigbird/blob/main/notebooks/infer-bigbird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference 🤗's BigBird

In [ ]:
%%capture
!pip3 install git+https://github.com/huggingface/transformers@master
!pip3 install sentencepiece
!pip3 install flax

## Flax BigBird

In [2]:
import jax.numpy as jnp
import jax
from transformers import FlaxBigBirdForQuestionAnswering, BigBirdTokenizerFast

In [3]:
model_id = "vasudevgupta/flax-bigbird-natural-questions"
model = FlaxBigBirdForQuestionAnswering.from_pretrained(model_id, block_size=16, num_random_blocks=3)
tokenizer = BigBirdTokenizerFast.from_pretrained(model_id)

print(jax.devices())

Some weights of the model checkpoint at vasudevgupta/flax-bigbird-natural-questions were not used when initializing FlaxBigBirdForQuestionAnswering: {('bert', 'pooler', 'bias'), ('cls', 'bias'), ('cls', 'kernel'), ('bert', 'pooler', 'kernel')}
- This IS expected if you are initializing FlaxBigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



[GpuDevice(id=0, process_index=0)]


In [9]:
context = "BigBird Pegasus just landed! Thanks to Vasudev Gupta, BigBird Pegasus from Google AI is merged into HuggingFace Transformers. Check it out today!!!"

In [7]:
def get_answer(question, context):
    @jax.jit
    def forward(**encoding):
      return model(**encoding)

    encoding = tokenizer(question, context, return_tensors="jax", max_length=256, padding="max_length", truncation=True)
    start_scores, end_scores = forward(**encoding).to_tuple()

    # Let's take the most likely token using `argmax` and retrieve the answer
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    answer_tokens = all_tokens[jnp.argmax(start_scores): jnp.argmax(end_scores)+1]
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

    return answer

In [10]:
question = "Who added BigBird to HuggingFace Transformers?"
get_answer(question, context)

'Vasudev Gupta'

## 🤗's `BigBirdModel`

In [ ]:
import torch
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizerFast

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
model_id = "google/bigbird-base-trivia-itc"
model = BigBirdForQuestionAnswering.from_pretrained(model_id, block_size=16, num_random_blocks=3).to(device)
tokenizer = BigBirdTokenizerFast.from_pretrained(model_id)

In [ ]:
context = "🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch. Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset"

In [ ]:
def get_answer(question, context):
    encoding = tokenizer(question, context, return_tensors="pt", max_length=256, padding="max_length", truncation=True)
    input_ids = encoding.input_ids.to(device)
    attention_mask = encoding.attention_mask.to(device)

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=input_ids, attention_mask=attention_mask).to_tuple()

    # Let's take the most likely token using `argmax` and retrieve the answer
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    answer_tokens = all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1]
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

    return answer

In [ ]:
question = "How many pretrained models are available in 🤗 Transformers?"
get_answer(question, context)

'32'

## 🤗's `BigBirdPegasus`

In [ ]:
import torch
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
model_id = "vasudevgupta/bigbird-pegasus-large-pubmed"
model = BigBirdPegasusForConditionalGeneration.from_pretrained(model_id, block_size=16, num_random_blocks=3).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
article = """There have been a number of interesting attempts, that were aimed at alleviating the quadratic dependency of Transformers, which can broadly categorized into two directions. First line of work embraces the length limitation and develops method around it. Simplest methods in this category just employ sliding window [93], but in general most work fits in the following general paradigm: using some other mechanism select a smaller subset of relevant contexts to feed in the transformer and optionally iterate, i.e. call transformer block multiple time with different contexts each time. Most prominently, SpanBERT [42], ORQA [54], REALM [34], RAG [57] have achieved strong performance for different tasks. However, it is worth noting that these methods often require significant engineering efforts (like back prop through large scale nearest neighbor search) and are hard to train. Second line of work questions if full attention is essential and have tried to come up with approaches that do not require full attention, thereby reducing the memory and computation requirements. Prominently, Dai et al. [21], Sukhbaatar et al. [82], Rae et al. [74] have proposed auto-regresive models that work well for left-to-right language modeling but suffer in tasks which require bidirectional context. Child et al. [16] proposed a sparse model that reduces the complexity to O(net al. [49] further reduced the complexity to O(n log(n)) by using LSH to compute nearest neighbors. Ye et al. [103] proposed binary partitions of the data where as Qiu et al. [73] reduced complexity by using block sparsity. Recently, Longformer [8] introduced a localized sliding window based mask with few global mask to reduce computation and extended BERT to longer sequence based tasks. Finally, our work is closely related to and built on the work of Extended Transformers Construction [4]. This work was designed to encode structure in text for transformers. The idea of global tokens was used extensively by them to achieve their goals. Our theoretical work can be seen as providing a justification for the success of these models as well. It is important to note that most of the aforementioned methods are heuristic based and empirically are not as versatile and robust as the original transformer, i.e. the same architecture do not attain SoTA on multiple standard benchmarks. (There is one exception of Longformer which we include in all our comparisons, see App. E.3 for a more detailed comparison). Moreover, these approximations do not come with theoretical guarantees."""

In [ ]:
article = """The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and

Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and
has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head
attention and the parameter-free position representation and became the other person involved in nearly every
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and
tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and
efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and
implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating
our research.
y
Work performed while at Google Brain.
z
Work performed while at Google Research.
31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.
transduction problems such as language modeling and machine translation [35,2,5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
statesh
t
, as a function of the previous hidden stateh
t1
and the input for positiont. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [21] and conditional
computation [32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2,19]. In all but a few cases [27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2 Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and output without using sequence-
aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [17, 18] and [9].
3 Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure [5,2,35].
Here, the encoder maps an input sequence of symbol representations(x
1
;:::;x
n
)to a sequence
of continuous representationsz= (z
1
;:::;z
n
). Givenz, the decoder then generates an output
sequence(y
1
;:::;y
m
)of symbols one element at a time. At each step the model is auto-regressive
[10], consuming the previously generated symbols as additional input when generating the next.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
Figure 1: The Transformer - model architecture.
3.1 Encoder and Decoder Stacks
Encoder:The encoder is composed of a stack ofN= 6identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [11] around each of
the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is
LayerNorm(x+ Sublayer(x)), whereSublayer(x)is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimensiond
model
= 512.
Decoder:
The decoder is also composed of a stack ofN= 6identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for positionican depend only on the known outputs at positions less thani.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
of the values, where the weight assigned to each value is computed by a compatibility function of the
query with the corresponding key.
Scaled Dot-Product AttentionMulti-Head Attention
Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
attention layers running in parallel.
3.2.1 Scaled Dot-Product Attention
We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of
queries and keys of dimensiond
k
, and values of dimensiond
v
. We compute the dot products of the
query with all keys, divide each by
p
d
k
, and apply a softmax function to obtain the weights on the
values.
In practice, we compute the attention function on a set of queries simultaneously, packed together
into a matrixQ. The keys and values are also packed together into matricesKandV. We compute
the matrix of outputs as:
Attention(Q;K;V) = softmax(
QK
T
p
d
k
)V(1)
The two most commonly used attention functions are additive attention [2], and dot-product (multi-
plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor
of
1
p
d
k
. Additive attention computes the compatibility function using a feed-forward network with
a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is
much faster and more space-efficient in practice, since it can be implemented using highly optimized
matrix multiplication code.
While for small values ofd
k
the two mechanisms perform similarly, additive attention outperforms
dot product attention without scaling for larger values ofd
k
[3]. We suspect that for large values of
d
k
, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients
4
. To counteract this effect, we scale the dot products by
1
p
d
k
.
3.2.2 Multi-Head Attention
Instead of performing a single attention function withd
model
-dimensional keys, values and queries,
we found it beneficial to linearly project the queries, keys and valueshtimes with different, learned
linear projections tod
k
,d
k
andd
v
dimensions, respectively. On each of these projected versions of
queries, keys and values we then perform the attention function in parallel, yieldingd
v
-dimensional
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
4
To illustrate why the dot products get large, assume that the components ofqandkare independent random
variables with mean0and variance1. Then their dot product,qk=
P
d
k
i=1
q
i
k
i
, has mean0and varianced
k
.
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhibits this.
MultiHead(Q;K;V) = Concat(head
1
;:::;head
h
)W
O
wherehead
i
= Attention(QW
Q
i
;KW
K
i
;VW
V
i
)
Where the projections are parameter matricesW
Q
i
2R
d
model
d
k
,W
K
i
2R
d
model
d
k
,W
V
i
2R
d
model
d
v
andW
O
2R
hd
v
d
model
.
In this work we employh= 8parallel attention layers, or heads. For each of these we use
d
k
=d
v
=d
model
=h= 64. Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.
3.2.3 Applications of Attention in our Model
The Transformer uses multi-head attention in three different ways:

In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].
The encoder contains self-attention layers. In a self-attention layer all of the keys, values
and queries come from the same place, in this case, the output of the previous layer in the
encoder. Each position in the encoder can attend to all positions in the previous layer of the
encoder.

Similarly, self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information flow in the decoder to preserve the auto-regressive property. We implement this
inside of scaled dot-product attention by masking out (setting to1) all values in the input
of the softmax which correspond to illegal connections. See Figure 2.
3.3 Position-wise Feed-Forward Networks
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully
connected feed-forward network, which is applied to each position separately and identically. This
consists of two linear transformations with a ReLU activation in between.
FFN(x) = max(0;xW
1
+b
1
)W
2
+b
2
(2)
While the linear transformations are the same across different positions, they use different parameters
from layer to layer. Another way of describing this is as two convolutions with kernel size 1.
The dimensionality of input and output isd
model
= 512, and the inner-layer has dimensionality
d
ff
= 2048.
3.4 Embeddings and Softmax
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimensiond
model
. We also use the usual learned linear transfor-
mation and softmax function to convert the decoder output to predicted next-token probabilities. In
our model, we share the same weight matrix between the two embedding layers and the pre-softmax
linear transformation, similar to [30]. In the embedding layers, we multiply those weights by
p
d
model
.
3.5 Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations
for different layer types.nis the sequence length,dis the representation dimension,kis the kernel
size of convolutions andrthe size of the neighborhood in restricted self-attention.
Layer Type Complexity per Layer Sequential Maximum Path Length
Operations
Self-AttentionO(n
2
d)O(1)O(1)
RecurrentO(nd
2
)O(n)O(n)
ConvolutionalO(knd
2
)O(1)O(log
k
(n))
Self-Attention (restricted)O(rnd)O(1)O(n=r)
tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimensiond
model
as the embeddings, so that the two can be summed. There are many choices of positional encodings,
learned and fixed [9].
In this work, we use sine and cosine functions of different frequencies:
PE
(pos;2i)
=sin(pos=10000
2i=d
model
)
PE
(pos;2i+1)
=cos(pos=10000
2i=d
model
)
whereposis the position andiis the dimension. That is, each dimension of the positional encoding
corresponds to a sinusoid. The wavelengths form a geometric progression from2to100002. We
chose this function because we hypothesized it would allow the model to easily learn to attend by
relative positions, since for any fixed offsetk,PE
pos+k
can be represented as a linear function of
PE
pos
.
We also experimented with using learned positional embeddings [9] instead, and found that the two
versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version
because it may allow the model to extrapolate to sequence lengths longer than the ones encountered
during training.
4 Why Self-Attention
In this section we compare various aspects of self-attention layers to the recurrent and convolu-
tional layers commonly used for mapping one variable-length sequence of symbol representations
(x
1
;:::;x
n
)to another sequence of equal length(z
1
;:::;z
n
), withx
i
;z
i
2R
d
, such as a hidden
layer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we
consider three desiderata.
One is the total computational complexity per layer. Another is the amount of computation that can
be parallelized, as measured by the minimum number of sequential operations required.
The third is the path length between long-range dependencies in the network. Learning long-range
dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the
ability to learn such dependencies is the length of the paths forward and backward signals have to
traverse in the network. The shorter these paths between any combination of positions in the input
and output sequences, the easier it is to learn long-range dependencies [12]. Hence we also compare
the maximum path length between any two input and output positions in networks composed of the
different layer types.
As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially
executed operations, whereas a recurrent layer requiresO(n)sequential operations. In terms of
computational complexity, self-attention layers are faster than recurrent layers when the sequence
lengthnis smaller than the representation dimensionalityd, which is most often the case with
sentence representations used by state-of-the-art models in machine translations, such as word-piece
[38] and byte-pair [31] representations. To improve computational performance for tasks involving
very long sequences, self-attention could be restricted to considering only a neighborhood of sizerin
the input sequence centered around the respective output position. This would increase the maximum
path length toO(n=r). We plan to investigate this approach further in future work.
A single convolutional layer with kernel widthk < ndoes not connect all pairs of input and output
positions. Doing so requires a stack ofO(n=k)convolutional layers in the case of contiguous kernels,
orO(log
k
(n))in the case of dilated convolutions [18], increasing the length of the longest paths
between any two positions in the network. Convolutional layers are generally more expensive than
recurrent layers, by a factor ofk. Separable convolutions [6], however, decrease the complexity
considerably, toO(knd+nd
2
). Even withk=n, however, the complexity of a separable
convolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer,
the approach we take in our model.
As side benefit, self-attention could yield more interpretable models. We inspect attention distributions
from our models and present and discuss examples in the appendix. Not only do individual attention
heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic
and semantic structure of the sentences.
5 Training
This section describes the training regime for our models.
5.1 Training Data and Batching
We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million
sentence pairs. Sentences were encoded using byte-pair encoding [3], which has a shared source-
target vocabulary of about 37000 tokens. For English-French, we used the significantly larger WMT
2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece
vocabulary [38]. Sentence pairs were batched together by approximate sequence length. Each training
batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000
target tokens.
5.2 Hardware and Schedule
We trained our models on one machine with 8 NVIDIA P100 GPUs. For our base models using
the hyperparameters described throughout the paper, each training step took about 0.4 seconds. We
trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the
bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps
(3.5 days).
5.3 Optimizer
We used the Adam optimizer [20] with
1
= 0:9,
2
= 0:98and= 10
9
. We varied the learning
rate over the course of training, according to the formula:
lrate=d
0:5
model
min(step_num
0:5
;step_numwarmup_steps
1:5
)(3)
This corresponds to increasing the learning rate linearly for the firstwarmup_stepstraining steps,
and decreasing it thereafter proportionally to the inverse square root of the step number. We used
warmup_steps= 4000.
5.4 Regularization
We employ three types of regularization during training:
Residual Dropout
We apply dropout [33] to the output of each sub-layer, before it is added to the
sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of
P
drop
= 0:1.
Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the
English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
Model
BLEU Training Cost (FLOPs)
EN-DE EN-FR EN-DE EN-FR
ByteNet [18] 23.75
Deep-Att + PosUnk [39] 39.21:010
20
GNMT + RL [38] 24.6 39.922:310
19
1:410
20
ConvS2S [9] 25.16 40.469:610
18
1:510
20
MoE [32] 26.03 40.562:010
19
1:210
20
Deep-Att + PosUnk Ensemble [39] 40.48:010
20
GNMT + RL Ensemble [38] 26.30 41.161:810
20
1:110
21
ConvS2S Ensemble [9] 26.3641.297:710
19
1:210
21
Transformer (base model) 27.3 38.13:310
18
Transformer (big)28.4 41.82:310
19
Label SmoothingDuring training, we employed label smoothing of value
ls
= 0:1[36]. This
hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.
6 Results
6.1 Machine Translation
On the WMT 2014 English-to-German translation task, the big transformer model (Transformer (big)
in Table 2) outperforms the best previously reported models (including ensembles) by more than2:0
BLEU, establishing a new state-of-the-art BLEU score of28:4. The configuration of this model is
listed in the bottom line of Table 3. Training took3:5days on8P100 GPUs. Even our base model
surpasses all previously published models and ensembles, at a fraction of the training cost of any of
the competitive models.
On the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of41:0,
outperforming all of the previously published single models, at less than1=4the training cost of the
previous state-of-the-art model. The Transformer (big) model trained for English-to-French used
dropout rateP
drop
= 0:1, instead of0:3.
For the base models, we used a single model obtained by averaging the last 5 checkpoints, which
were written at 10-minute intervals. For the big models, we averaged the last 20 checkpoints. We
used beam search with a beam size of4and length penalty= 0:6[38]. These hyperparameters
were chosen after experimentation on the development set. We set the maximum output length during
inference to input length +50, but terminate early when possible [38].
Table 2 summarizes our results and compares our translation quality and training costs to other model
architectures from the literature. We estimate the number of floating point operations used to train a
model by multiplying the training time, the number of GPUs used, and an estimate of the sustained
single-precision floating-point capacity of each GPU
5
.
6.2 Model Variations
To evaluate the importance of different components of the Transformer, we varied our base model
in different ways, measuring the change in performance on English-to-German translation on the
development set, newstest2013. We used beam search as described in the previous section, but no
checkpoint averaging. We present these results in Table 3.
In Table 3 rows (A), we vary the number of attention heads and the attention key and value dimensions,
keeping the amount of computation constant, as described in Section 3.2.2. While single-head
attention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads.
5
We used values of 2.8, 3.7, 6.0 and 9.5 TFLOPS for K80, K40, M40 and P100, respectively.
Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base
model. All metrics are on the English-to-German translation development set, newstest2013. Listed
perplexities are per-wordpiece, according to our byte-pair encoding, and should not be compared to
per-word perplexities.
N d
model
d
ff
h d
k
d
v
P
drop

ls
trainPPL BLEU params
steps(dev) (dev)10
6
base6 512 2048 8 64 64 0.1 0.1 100K4.92 25.8 65
(A)
1 512 5125.29 24.9
4 128 1285.00 25.5
16 32 324.91 25.8
32 16 165.01 25.4
(B)
165.16 25.1 58
325.01 25.4 60
(C)
26.11 23.7 36
45.19 25.3 50
84.88 25.5 80
256 32 325.75 24.5 28
1024 128 1284.66 26.0 168
10245.12 25.4 53
40964.75 26.2 90
(D)
0.05.77 24.6
0.24.95 25.5
0.04.67 25.3
0.25.47 25.7
(E)positional embedding instead of sinusoids4.92 25.7
big6 1024 4096 16 0.3 300K4.33 26.4213
Table 4: The Transformer generalizes well to English constituency parsing (Results are on Section 23
of WSJ)
ParserTrainingWSJ 23 F1
Vinyals & Kaiser el al. (2014) [37]WSJ only, discriminative88.3
Petrov et al. (2006) [29]
WSJ only, discriminative90.4
Zhu et al. (2013) [40]WSJ only, discriminative90.4
Dyer et al. (2016) [8]WSJ only, discriminative91.7
Transformer (4 layers)WSJ only, discriminative91.3
Zhu et al. (2013) [40]semi-supervised91.3
Huang & Harper (2009) [14]semi-supervised91.3
McClosky et al. (2006) [26]semi-supervised92.1
Vinyals & Kaiser el al. (2014) [37]semi-supervised92.1
Transformer (4 layers)semi-supervised92.7
Luong et al. (2015) [23]multi-task93.0
Dyer et al. (2016) [8]generative93.3
In Table 3 rows (B), we observe that reducing the attention key sized
k
hurts model quality. This
suggests that determining compatibility is not easy and that a more sophisticated compatibility
function than dot product may be beneficial. We further observe in rows (C) and (D) that, as expected,
bigger models are better, and dropout is very helpful in avoiding over-fitting. In row (E) we replace our
sinusoidal positional encoding with learned positional embeddings [9], and observe nearly identical
results to the base model.
6.3 English Constituency Parsing
To evaluate if the Transformer can generalize to other tasks we performed experiments on English
constituency parsing. This task presents specific challenges: the output is subject to strong structural
constraints and is significantly longer than the input. Furthermore, RNN sequence-to-sequence
models have not been able to attain state-of-the-art results in small-data regimes [37].
We trained a 4-layer transformer withd
model
= 1024on the Wall Street Journal (WSJ) portion of the
Penn Treebank [25], about 40K training sentences. We also trained it in a semi-supervised setting,
using the larger high-confidence and BerkleyParser corpora from with approximately 17M sentences
[37]. We used a vocabulary of 16K tokens for the WSJ only setting and a vocabulary of 32K tokens
for the semi-supervised setting.
We performed only a small number of experiments to select the dropout, both attention and residual
(section 5.4), learning rates and beam size on the Section 22 development set, all other parameters
remained unchanged from the English-to-German base translation model. During inference, we
increased the maximum output length to input length +300. We used a beam size of21and= 0:3
for both WSJ only and the semi-supervised setting.
Our results in Table 4 show that despite the lack of task-specific tuning our model performs sur-
prisingly well, yielding better results than all previously reported models with the exception of the
Recurrent Neural Network Grammar [8].
In contrast to RNN sequence-to-sequence models [37], the Transformer outperforms the Berkeley-
Parser [29] even when training only on the WSJ training set of 40K sentences.
7 Conclusion
In this work, we presented the Transformer, the first sequence transduction model based entirely on
attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with
multi-headed self-attention.
For translation tasks, the Transformer can be trained significantly faster than architectures based
on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014
English-to-French translation tasks, we achieve a new state of the art. In the former task our best
model outperforms even all previously reported ensembles.
We are excited about the future of attention-based models and plan to apply them to other tasks. We
plan to extend the Transformer to problems involving input and output modalities other than text and
to investigate local, restricted attention mechanisms to efficiently handle large inputs and outputs
such as images, audio and video. Making generation less sequential is another research goals of ours.
The code we used to train and evaluate our models is available athttps://github.com/
tensorflow/tensor2tensor.
AcknowledgementsWe are grateful to Nal Kalchbrenner and Stephan Gouws for their fruitful
comments, corrections and inspiration.
References
[1]Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E Hinton. Layer normalization.arXiv preprint
arXiv:1607.06450, 2016.
[2]
Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly
learning to align and translate.CoRR, abs/1409.0473, 2014.
[3]
Denny Britz, Anna Goldie, Minh-Thang Luong, and Quoc V. Le. Massive exploration of neural
machine translation architectures.CoRR, abs/1703.03906, 2017.
[4]
Jianpeng Cheng, Li Dong, and Mirella Lapata. Long short-term memory-networks for machine
reading.arXiv preprint arXiv:1601.06733, 2016.
[5]
Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk,
and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical
machine translation.CoRR, abs/1406.1078, 2014.
[6]Francois Chollet. Xception: Deep learning with depthwise separable convolutions.arXiv
preprint arXiv:1610.02357, 2016.
[7]Junyoung Chung, Çaglar Gülçehre, Kyunghyun Cho, and Yoshua Bengio. Empirical evaluation
of gated recurrent neural networks on sequence modeling.CoRR, abs/1412.3555, 2014.
[8]Chris Dyer, Adhiguna Kuncoro, Miguel Ballesteros, and Noah A. Smith. Recurrent neural
network grammars. InProc. of NAACL, 2016.
[9]Jonas Gehring, Michael Auli, David Grangier, Denis Yarats, and Yann N. Dauphin. Convolu-
tional sequence to sequence learning.arXiv preprint arXiv:1705.03122v2, 2017.
[10]
Alex Graves. Generating sequences with recurrent neural networks.arXiv preprint
arXiv:1308.0850, 2013.
[11]Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. Deep residual learning for im-
age recognition. InProceedings of the IEEE Conference on Computer Vision and Pattern
Recognition, pages 770–778, 2016.
[12]Sepp Hochreiter, Yoshua Bengio, Paolo Frasconi, and Jürgen Schmidhuber. Gradient flow in
recurrent nets: the difficulty of learning long-term dependencies, 2001.
[13]
Sepp Hochreiter and Jürgen Schmidhuber. Long short-term memory.Neural computation,
9(8):1735–1780, 1997.
[14]Zhongqiang Huang and Mary Harper. Self-training PCFG grammars with latent annotations
across languages. InProceedings of the 2009 Conference on Empirical Methods in Natural
Language Processing, pages 832–841. ACL, August 2009.
[15]Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, and Yonghui Wu. Exploring
the limits of language modeling.arXiv preprint arXiv:1602.02410, 2016.
[16]
Łukasz Kaiser and Samy Bengio. Can active memory replace attention? InAdvances in Neural
Information Processing Systems, (NIPS), 2016.
[17]Łukasz Kaiser and Ilya Sutskever. Neural GPUs learn algorithms. InInternational Conference
on Learning Representations (ICLR), 2016.
[18]Nal Kalchbrenner, Lasse Espeholt, Karen Simonyan, Aaron van den Oord, Alex Graves, and Ko-
ray Kavukcuoglu. Neural machine translation in linear time.arXiv preprint arXiv:1610.10099v2,
2017.
[19]Yoon Kim, Carl Denton, Luong Hoang, and Alexander M. Rush. Structured attention networks.
InInternational Conference on Learning Representations, 2017.
[20]Diederik Kingma and Jimmy Ba. Adam: A method for stochastic optimization. InICLR, 2015.
[21]Oleksii Kuchaiev and Boris Ginsburg. Factorization tricks for LSTM networks.arXiv preprint
arXiv:1703.10722, 2017.
[22]Zhouhan Lin, Minwei Feng, Cicero Nogueira dos Santos, Mo Yu, Bing Xiang, Bowen
Zhou, and Yoshua Bengio. A structured self-attentive sentence embedding.arXiv preprint
arXiv:1703.03130, 2017.
[23]Minh-Thang Luong, Quoc V. Le, Ilya Sutskever, Oriol Vinyals, and Lukasz Kaiser. Multi-task
sequence to sequence learning.arXiv preprint arXiv:1511.06114, 2015.
[24]Minh-Thang Luong, Hieu Pham, and Christopher D Manning. Effective approaches to attention-
based neural machine translation.arXiv preprint arXiv:1508.04025, 2015.
[25]Mitchell P Marcus, Mary Ann Marcinkiewicz, and Beatrice Santorini. Building a large annotated
corpus of english: The penn treebank.Computational linguistics, 19(2):313–330, 1993.
[26]David McClosky, Eugene Charniak, and Mark Johnson. Effective self-training for parsing. In
Proceedings of the Human Language Technology Conference of the NAACL, Main Conference,
pages 152–159. ACL, June 2006.
[27]Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention
model. InEmpirical Methods in Natural Language Processing, 2016.
[28]
Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive
summarization.arXiv preprint arXiv:1705.04304, 2017.
[29]
Slav Petrov, Leon Barrett, Romain Thibaux, and Dan Klein. Learning accurate, compact,
and interpretable tree annotation. InProceedings of the 21st International Conference on
Computational Linguistics and 44th Annual Meeting of the ACL, pages 433–440. ACL, July
2006.
[30]
Ofir Press and Lior Wolf. Using the output embedding to improve language models.arXiv
preprint arXiv:1608.05859, 2016.
[31]
Rico Sennrich, Barry Haddow, and Alexandra Birch. Neural machine translation of rare words
with subword units.arXiv preprint arXiv:1508.07909, 2015.
[32]Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton,
and Jeff Dean. Outrageously large neural networks: The sparsely-gated mixture-of-experts
layer.arXiv preprint arXiv:1701.06538, 2017.
[33]
Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-
nov. Dropout: a simple way to prevent neural networks from overfitting.Journal of Machine
Learning Research, 15(1):1929–1958, 2014.
[34]Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, and Rob Fergus. End-to-end memory
networks. In C. Cortes, N. D. Lawrence, D. D. Lee, M. Sugiyama, and R. Garnett, editors,
Advances in Neural Information Processing Systems 28, pages 2440–2448. Curran Associates,
Inc., 2015.
[35]Ilya Sutskever, Oriol Vinyals, and Quoc VV Le. Sequence to sequence learning with neural
networks. InAdvances in Neural Information Processing Systems, pages 3104–3112, 2014.
[36]Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jonathon Shlens, and Zbigniew Wojna.
Rethinking the inception architecture for computer vision.CoRR, abs/1512.00567, 2015.
[37]Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton. Grammar as a foreign language. In
Advances in Neural Information Processing Systems, 2015.
[38]Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang
Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine
translation system: Bridging the gap between human and machine translation.arXiv preprint
arXiv:1609.08144, 2016.
[39]Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation.CoRR, abs/1606.04199, 2016.
[40]Muhua Zhu, Yue Zhang, Wenliang Chen, Min Zhang, and Jingbo Zhu. Fast and accurate
shift-reduce constituent parsing. InProceedings of the 51st Annual Meeting of the ACL (Volume
1: Long Papers), pages 434–443. ACL, August 2013.
Attention Visualizations
Input-Input Layer5
It
is
inthis
spirit
that
a
majority
ofAmerican
governments
have
passed
newlaws
since
2009
making
the
registration
or
voting
process
more
dif
ficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
dif
ficult
.
<EOS>
<pad><pad>
<pad>
<pad>
<pad>
<pad>
Figure 3: An example of the attention mechanism following long-distance dependencies in the
encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of
the verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for
the word ‘making’. Different colors represent different heads. Best viewed in color.
Input-Input Layer5
The
Law
willnever
be
perfect
,
but
itsapplication
should
be
just
-this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Input-Input Layer5
The
Law
willnever
be
perfect
,
but
itsapplication
should
be
just
-this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top:
Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5
and 6. Note that the attentions are very sharp for this word.
Input-Input Layer5
The
Law
willnever
be
perfect
,
but
itsapplication
should
be
just
-this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Input-Input Layer5
The
Law
willnever
be
perfect
,
but
itsapplication
should
be
just
-this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the
sentence. We give two such examples above, from two different heads from the encoder self-attention
at layer 5 of 6. The heads clearly learned to perform different tasks."""

In [ ]:
inputs = tokenizer(article, max_length=4096, padding="max_length", return_tensors="pt", truncation=True)
inputs = {k: inputs[k].to(device) for k in inputs}

In [ ]:
outputs = model.generate(**inputs, max_length=256)

In [ ]:
tokenizer.batch_decode(outputs)[0]

'<s> the study of programming languages and the study of the evolution of programming language is one of the areas of the life sciences that has the potential to impact the way we think about problems, solve problems, and improve the quality of life for millions of people.<n> one of the major goals of the programming community is to create programming languages that can be used for a broad range of applications.<n> this will require an understanding of the programming environment, the characteristics of the programming environment, and the evolution of the programming language over time.<n> we are at the beginning of a new era in the study of the evolution of programming languages, and our goal here is to briefly introduce the reader to some of the concepts that are new to this area of study.</s>'